In [ ]:
# import necessary libraries
import airflow
import requests
from funcy import project

from airflow import DAG
from airflow.operators.python import PythonOperator


# define DAG
mlb_dag=DAG(
    dag_id="mlb_info",
    start_date=airflow.utils.dates.days_ago(1),
    schedule_interval= None,
    #schedule_interval= @weekly
    # if you want to schedule weekly stats
)



In [ ]:
# tasks
def _get_team():
    response = requests.get("http://lookup-service-prod.mlb.com/json/named.team_all_season.bam?sports_code='mlb'&season='2023'")
    teams = response.json()['team_all_season']['queryResults']['row']
    print(f"MLB lookup service API response code for team data: {response.status_code}")

    # find dodgers team id
    team_list = list(x['team_id'] for x in teams if x['name_display_full'] == "Los Angeles Dodgers")
    print(team_list[0])


In [ ]:
'''RETURNS THE 40 MAN ROSTER'''
def _get_players(**context):
    # fetch team values from team task
    team_id = context['task_instance'].xcom_pull(task_ids='get_team')
    # x.com_pull()' used to pull a list of return values from one or multiple Airflow tasks

    # get player data from API
    response = requests.get(f"https://lookup-service-prod.mlb.com/json/named.roster_40.bam?team_id='{team_id}'")
    players = response.json()['roster_40']['queryResults']['row']
    print(f"MLB lookup service API response code for players data: {response.status_code}")

    # filter to left-handed batters
    players_list = list(x for x in players if x['bats'] == 'L')
    return players_list


In [ ]:
def _write_file():
    print("_write_file task is running.")

# write operators
get_team = PythonOperator(
    task_id = "get_team",
    python_callable=_get_team,
    dag=mlb_dag,
)

get_players = PythonOperator(
    task_id = "get_players",
    python_callable=_get_players,
    dag=mlb_dag,
)

'''get_games = PythonOperator(
    task_id = "get_games",
    python_callable=_get_games,
    dag=mlb_dag,
)'''

write_file = PythonOperator(
    task_id = "write_file",
    python_callable=_write_file,
    dag=mlb_dag,
)

# structure dag
get_team >> [get_players, get_team] >> write_file